In [1]:
import numpy as np 
import csv 
import pandas as pd
from scipy.linalg import dft
from scipy import fft
import matplotlib.pyplot as plt 
import bitarray
import pdb


In [2]:
channelResponse = pd.read_csv('channel.csv', header = None)
channelResponse = channelResponse.to_numpy()
FILE_NAME = 'file9.csv'
file = pd.read_csv(FILE_NAME, header = None)

In [8]:
M = len(channelResponse) #FIR filter horizon 
N = 1024 # number of OFDM subcarriers
L = 32 #length of cyclic prefix 
mu = 2 #bits per constellation symbol 

In [4]:
channelZeros = np.zeros(N-M)
channelResponse = np.append(channelResponse, channelZeros)
H_exact = fft(channelResponse, n = N)

In [13]:
def demapping_symbol(symbol):
    first_bit = 0 if symbol.imag >= 0 else 1
    second_bit = 0 if symbol.real >= 0 else 1
    return first_bit, second_bit

def demapping(sequence):
    output = []
    for f in sequence:
        first_bit, second_bit = demapping_symbol(f)
        output.append(first_bit)
        output.append(second_bit)
    return output

def split_signal(file, N, L):
    return np.array_split(file, len(file)/1056)

def disregard_cyclic_prefix(splitted_file, L):
    return [list(w[L:][0]) for w in splitted_file]

def apply_fft(splitted_file, N):
    fft_blocks = [fft(block, N) for block in splitted_file]
    return fft_blocks 

def equalize(fft_blocks, H):
    equalized_fft_blocks = [np.divide(r, H) for r in fft_blocks]
    return equalized_fft_blocks 


def disregard_freq_bins(equalized_fft_blocks):
    return [d[1:512] for d in equalized_fft_blocks]

def fft_file(file, H, N, L):
    """
    returns fft of the file, before ML mapping, fft is in chunks of OFDM blocks
    """
    split_signal_1056 = split_signal(file, N, L)
    split_signal_1024 = disregard_cyclic_prefix(split_signal_1056, L)
    fft_blocks = apply_fft(split_signal_1024, N)
    equalized_fft_blocks = equalize(fft_blocks, H_exact)
    return disregard_freq_bins(equalized_fft_blocks)

def demap_fft(fft_signal):
    bits = []
    for d in fft_signal:
        bits.extend(demapping(d))
    return bits 

def bits_to_bytes(bits):
    bits_string = "".join((str(s) for s in bits))
    output_bytes = [bits_string[i : i + 8] for i in range(0, len(bits_string), 8)]
    output_bytes = bytearray([int(i, 2) for i in output_bytes])
    return output_bytes 

def file_to_decoded_bytes(file, H, N, L):
    fft_signal_unmaped = fft_file(file, H, N, L)
    fft_signal_demapped = demap_fft(fft_signal_unmaped)
    bytes_output = bits_to_bytes(fft_signal_demapped)
    return bytes_output

In [14]:
output_bytes = file_to_decoded_bytes(file, H_exact, N, L)
print(output_bytes[:50])

bytearray(b'files/9103751287.wav\x0058584\x00RIFF\xd0\xe4\x00\x00WAVEfmt \x10\x00\x00\x00\x01\x00\x01')


In [16]:
del output_bytes[:len('files/9103751287.wav\x0058584\x00')]
if 'tiff' in 'files/9103751287.wav\x0058584\x00' or True:
    with open(f"output_{FILE_NAME[:-4]}.wav", "w+b") as f:
        f.write(output_bytes)